#  CS 522 - Advanced Data Mining - Spring 2016 - Final Project

#                Gender Classification using Twitter Feeds

# Step - 3) Tokenize the tweets.

In [55]:
# Import all required packages
import pickle
import re
from itertools import product
import nltk
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter
from scipy.sparse import lil_matrix

In [103]:
from sampleClass14 import sampleClass14

In [104]:
sampleClass = sampleClass14()

In [40]:
# Load these tweets
tweets = pickle.load(open('tweets.pkl', 'rb'))
junk_tweets = pickle.load(open('junk_tweets.pkl', 'rb')) 

In [58]:
test_tweet = tweets[10]
print 'test tweet:\n\tdescr=%s\n\ttext=%s' % (test_tweet['user']['description'],test_tweet['text'])
print 'user name: \n %s' %(test_tweet['user']['name']) 

test tweet:
	descr=find your greatness • ROC CHEER&SOFTBALL • #GarmoStrong
	text=you SUCK 🍭
user name: 
 karlyn kelley


In [51]:
test_junk_tweet = junk_tweets[10]
print 'test junk tweet:\n\tdescr=%s\n\ttext=%s' % (test_junk_tweet['user']['description'], test_junk_tweet['text'])
print 'junk user name: \n %s' %(test_junk_tweet['user']['name']) 

test junk tweet:
	descr=Father, Husband, Coach, Trainer, Member of Omega Psi Phi, Mentor, Pro Natural Bodybuilder,Former NFL, CFL, and Arena League Football Player.
	text=Keep putting in work #swac #hbcu https://t.co/pdGaGY74ac
junk user name: 
 Coach Hicks


In [59]:
print stopwords.words('english')

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [60]:
import re
try:
    # Wide UCS-4 build
    myre = re.compile(u'['
        u'\U0001F300-\U0001F64F'
        u'\U0001F680-\U0001F6FF'
        u'\U0001f602'
        u'\u2600-\u26FF\u2700-\u27BF\ufe0f]+', 
        re.UNICODE)
except re.error:
    # Narrow UCS-2 build
    myre = re.compile(u'('
        u'\ud83c[\udf00-\udfff]|'
        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
        u'[\u2600-\u26FF\u2700-\u27BF])+', 
        re.UNICODE)
    
print myre.sub('',test_junk_tweet['user']['description'])

Father, Husband, Coach, Trainer, Member of Omega Psi Phi, Mentor, Pro Natural Bodybuilder,Former NFL, CFL, and Arena League Football Player.


In [64]:

def tokenize(string, lowercase, keep_punctuation, prefix,collapse_urls, collapse_mentions,collapse_stop_words):
    if not string:
        return []
    if lowercase:
        string = string.lower()
    tokens = []
    if collapse_urls:
        string = re.sub('http\S+', 'THIS_IS_A_URL', string)
    if collapse_mentions:
        string = re.sub('@\S+', 'THIS_IS_A_MENTION', string)
    if keep_punctuation:
        tokens = string.split()
    else:
        tokens = re.sub('\W+', ' ', string).split()
    if collapse_stop_words:
        tokens = list(set(tokens) - set(stopwords.words('english')))
    if prefix:
        tokens = ['%s%s' % (prefix, t) for t in tokens]
    return tokens


In [45]:
#five separate people almost hit my car in a matter of 30 seconds. 
#the students at Lamar need to take a driving course

In [61]:
print junk_tweets[10]['text']
print sampleClass.tokenize(junk_tweets[10]['text'], False, False, None,False, False,False)
print sampleClass.tokenize(junk_tweets[10]['text'], True, True, None,True, True,True)

Keep putting in work #swac #hbcu https://t.co/pdGaGY74ac
[u'Keep', u'putting', u'in', u'work', u'swac', u'hbcu', u'https', u't', u'co', u'pdGaGY74ac']
[u'work', u'keep', u'#hbcu', u'putting', u'#swac', u'THIS_IS_A_URL']


In [63]:
print sampleClass.tokenize(tweets[10]['text'], False, False, None,False, False,True)

[u'SUCK']


In [66]:
print tweets[10]['text']
print sampleClass.tokenize(tweets[10]['text'], False, False, None,False, False,False)
print sampleClass.tokenize(tweets[10]['text'], False, False, None,True, True,False)

you SUCK 🍭
[u'you', u'SUCK']
[u'you', u'SUCK']


In [67]:
def tweet2tokens(tweet, use_descr=True, lowercase=True,keep_punctuation=True, descr_prefix='d=',
                 collapse_urls=True, collapse_mentions=True, use_text=True,collapse_stop_words = True):
    tokens = []
    if use_text:
        tokens.extend(tokenize(tweet['text'], lowercase, keep_punctuation, None,collapse_urls, collapse_mentions,collapse_stop_words))
    if use_descr:
        tokens.extend(tokenize(tweet['user']['description'], lowercase,keep_punctuation, descr_prefix,collapse_urls, collapse_mentions,collapse_stop_words))
    return tokens

In [68]:
tweet2tokens(test_junk_tweet,True,True,True,None,True,True,True,True)

[u'work',
 u'keep',
 u'#hbcu',
 u'putting',
 u'#swac',
 u'THIS_IS_A_URL',
 u'arena',
 u'league',
 u'psi',
 u'natural',
 u'pro',
 u'phi,',
 u'player.',
 u'football',
 u'husband,',
 u'nfl,',
 u'trainer,',
 u'mentor,',
 u'member',
 u'father,',
 u'bodybuilder,former',
 u'coach,',
 u'omega',
 u'cfl,']

In [76]:
sampleClass.tweet2tokens(test_junk_tweet,True,True,True,None,True,True,True,True)

[u'work',
 u'keep',
 u'#hbcu',
 u'putting',
 u'#swac',
 u'THIS_IS_A_URL',
 u'arena',
 u'league',
 u'psi',
 u'natural',
 u'pro',
 u'phi,',
 u'player.',
 u'football',
 u'husband,',
 u'nfl,',
 u'trainer,',
 u'mentor,',
 u'member',
 u'father,',
 u'bodybuilder,former',
 u'coach,',
 u'omega',
 u'cfl,']

In [19]:
use_descr_opts = [True, False]
lowercase_opts = [True, False]
keep_punctuation_opts = [True, False]
descr_prefix_opts = ['d=', '']
url_opts = [True, False]
mention_opts = [True, False]
use_text_opts = [True, False]
stop_words_opts = [True, False]

argnames = ['use_descr', 'lower', 'punct', 'prefix', 'url', 'mention', 'use_text', 'collapse_stop_words']
option_iter = product(use_descr_opts, lowercase_opts,keep_punctuation_opts,descr_prefix_opts, url_opts,
                       mention_opts, use_text_opts,stop_words_opts)

# Write the output of various options to a file
f = open('options.txt', 'w')
for options in option_iter:
    f.write('\t'.join('%s=%s' % (name, opt) for name, opt in zip(argnames, options)))
    f.write('\n---------------------------\n')
    f.write(myre.sub('','    '.join(tweet2tokens(test_tweet, *options))))
    f.write('\n========================================================================\n')
f.close()

In [20]:
# Write the output of various options of junk tweets to a file
use_descr_opts = [True, False]
lowercase_opts = [True, False]
keep_punctuation_opts = [True, False]
descr_prefix_opts = ['d=', '']
url_opts = [True, False]
mention_opts = [True, False]
use_text_opts = [True, False]
stop_words_opts = [True, False]

argnames = ['use_descr', 'lower', 'punct', 'prefix', 'url', 'mention', 'use_text', 'collapse_stop_words']
option_iter = product(use_descr_opts, lowercase_opts,keep_punctuation_opts,descr_prefix_opts, url_opts,
                       mention_opts, use_text_opts,stop_words_opts)


f1 = open('options_junk_tweets.txt', 'w')
for options in option_iter:
    f1.write('\t'.join('%s=%s' % (name, opt) for name, opt in zip(argnames, options)))
    f1.write('\n---------------------------\n')
    f1.write(myre.sub('','    '.join(tweet2tokens(test_junk_tweet, *options))))
    f1.write('\n========================================================================\n')
f1.close()

In [77]:
print 'Total tweets we have so far = ',len(tweets)

Total tweets we have so far =  10000


In [80]:
tokens_list = [sampleClass.tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=False, descr_prefix='d=',
                            collapse_urls=True, collapse_mentions=True,collapse_stop_words=True)
              for t in tweets]

In [84]:
tokens_junk_list = [sampleClass.tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=False, descr_prefix='d=',
                            collapse_urls=True, collapse_mentions=True,collapse_stop_words=True)
              for t in junk_tweets]

In [83]:
print tokens_junk_list[10]

[u'swac', u'work', u'keep', u'putting', u'hbcu', u'THIS_IS_A_URL', u'd=trainer', u'd=league', u'd=phi', u'd=coach', u'd=natural', u'd=pro', u'd=nfl', u'd=father', u'd=cfl', u'd=arena', u'd=football', u'd=member', u'd=player', u'd=bodybuilder', u'd=psi', u'd=mentor', u'd=omega', u'd=former', u'd=husband']


In [86]:
# Create Vocabulary (A dictionary from term to index)

def make_vocabulary(tokens_list):
    vocabulary = defaultdict(lambda: len(vocabulary)) # Similar to vocabulary = defaultdict(int)
    for tokens in tokens_list:
        for token in tokens:
            vocabulary[token]
    print '%d unique terms in vocabulary' % len(vocabulary)
    return vocabulary

In [88]:
vocabulary = sampleClass.make_vocabulary(tokens_list)

28340 unique terms in vocabulary


In [87]:
# How big is vocabulary if there is no punctuation
vocabulary = make_vocabulary(tokens_list)

28340 unique terms in vocabulary


In [91]:
vocabulary_junk_tweets = sampleClass.make_vocabulary(tokens_junk_list)

1855 unique terms in vocabulary


In [31]:
# How big is vocabulary if we keep punctuation?
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=True, collapse_mentions=True,collapse_stop_words=True)
              for t in tweets]
print len(tokens_list)

vocabulary = make_vocabulary(tokens_list)
d = Counter(vocabulary)
print "Top 30 values in vocabulary are :" ,d.most_common(30)

10000
43019 unique terms in vocabulary
Top 30 values in vocabulary are : [(u'd=senojl', 43018), (u'd=keona', 43017), (u'd=1968.', 43016), (u'd=orchard', 43015), (u'trek.', 43014), (u'#max.', 43013), (u'spin', 43012), (u'wheels!', 43011), (u'letters', 43010), (u'd=mixx', 43009), (u'eder', 43008), (u'perez', 43007), (u'employees', 43006), (u'd=gfhs', 43005), (u'd=#proera47', 43004), (u'b4.da.$$', 43003), (u'caffiene', 43002), (u'jitters', 43001), (u'trains!"', 43000), (u'd=sandymadanatprTHIS_IS_A_MENTION', 42999), (u'destin', 42998), (u'grinding.', 42997), (u'd=astralis', 42996), (u'd=convincing', 42995), (u'd=1976.', 42994), (u'peachtree', 42993), (u'#moveslikejagger', 42992), (u'd=teenage', 42991), (u'd=mutant', 42990), (u'd=turtle!!!', 42989)]


In [94]:
# How big is vocabulary if we keep punctuation and urls?
tokens_list = [sampleClass.tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=True,collapse_stop_words=True)
              for t in tweets]
print len(tokens_list)

vocabulary = sampleClass.make_vocabulary(tokens_list)
d = Counter(vocabulary)
print "Top 30 values in vocabulary are :" ,d.most_common(30)

10000
47062 unique terms in vocabulary
Top 30 values in vocabulary are : [(u'politian', 47061), (u'cross-pollination?', 47060), (u'critic', 47059), (u'with...', 47058), (u'd=4.3.17', 47057), (u'#atl', 47056), (u"lol'd", 47055), (u'oversight', 47054), (u'demonstrative', 47053), (u'abdelrazik,', 47052), (u'case,', 47051), (u'fitting', 47050), (u'd=bestfriend\u2764\ufe0f', 47049), (u'place\U0001f629', 47048), (u'nervous', 47047), (u'd=savvy?', 47046), (u'world!', 47045), (u'tweeting,', 47044), (u'least!', 47043), (u'doctrine', 47042), (u'issue.', 47041), (u'talking!!', 47040), (u"d=vibin'", 47039), (u'automatic', 47038), (u'd=gmen,', 47037), (u'd=wiz,', 47036), (u'd=nats,', 47035), (u'd=terps,', 47034), (u'd=#onedematha', 47033), (u'unsupervised', 47032)]


In [96]:
# How big is vocabulary if we keep punctuation and urls and mentions?
tokens_list = [sampleClass.tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=False,collapse_stop_words=True)
              for t in tweets]
print len(tokens_list)

vocabulary = sampleClass.make_vocabulary(tokens_list)
d = Counter(vocabulary)
print "Top 30 values in vocabulary are :" ,d.most_common(30)

10000
53557 unique terms in vocabulary
Top 30 values in vocabulary are : [(u'politian', 53556), (u'cross-pollination?', 53555), (u'with...', 53554), (u'critic', 53553), (u'@mikesfilmtalk', 53552), (u'@copromote', 53551), (u'd=4.3.17', 53550), (u'#atl', 53549), (u'@lilrobzzz', 53548), (u"lol'd", 53547), (u'oversight', 53546), (u'@kuhn_eyesopen\u2019s', 53545), (u'demonstrative', 53544), (u'abdelrazik,', 53543), (u'case,', 53542), (u".@ararmaher's", 53541), (u'fitting', 53540), (u'd=bestfriend\u2764\ufe0f', 53539), (u'place\U0001f629', 53538), (u'nervous', 53537), (u'd=savvy?', 53536), (u'world!', 53535), (u'tweeting,', 53534), (u'least!', 53533), (u'@adriennesmith40', 53532), (u'doctrine', 53531), (u'issue.', 53530), (u'talking!!', 53529), (u"d=vibin'", 53528), (u'@cbeauregard8', 53527)]


In [98]:
# How big is vocabulary if we keep punctuation and urls and mentions and stop words?
tokens_list = [sampleClass.tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=False,collapse_stop_words=False)
              for t in tweets]


vocabulary = sampleClass.make_vocabulary(tokens_list)
d = Counter(vocabulary)
print "Top 30 values in vocabulary are :" ,d.most_common(30)

53816 unique terms in vocabulary
Top 30 values in vocabulary are : [(u'politian', 53815), (u'with...', 53814), (u'critic', 53813), (u'cross-pollination?', 53812), (u'@copromote', 53811), (u'@mikesfilmtalk', 53810), (u'd=4.3.17', 53809), (u'#atl', 53808), (u"lol'd", 53807), (u'@lilrobzzz', 53806), (u'oversight', 53805), (u'demonstrative', 53804), (u'@kuhn_eyesopen\u2019s', 53803), (u'abdelrazik,', 53802), (u'case,', 53801), (u".@ararmaher's", 53800), (u'fitting', 53799), (u'd=bestfriend\u2764\ufe0f', 53798), (u'place\U0001f629', 53797), (u'nervous', 53796), (u'd=savvy?', 53795), (u'least!', 53794), (u'tweeting,', 53793), (u'world!', 53792), (u'@adriennesmith40', 53791), (u'issue.', 53790), (u'doctrine', 53789), (u'talking!!', 53788), (u"d=vibin'", 53787), (u'@cbeauregard8', 53786)]


In [105]:
X1 = sampleClass.make_feature_matrix(tokens_list,vocabulary,len(tweets))
print X1.shape

(10000, 53816)


In [99]:
# Make a CSR feature matrix to map each tweet to the frequency of it token appearance
# X[i,j] is the frequency of term j in tweet i

def make_feature_matrix(tokens_list, vocabulary):
    X = lil_matrix((len(tweets), len(vocabulary)))
    for i, tokens in enumerate(tokens_list):
        for token in tokens:
            j = vocabulary[token]
            X[i,j] += 1
    return X.tocsr()

In [100]:
X = make_feature_matrix(tokens_list, vocabulary)
print 'shape of X:', X.shape

shape of X: (10000, 53816)
